In [6]:
# Loading data

import numpy as np
from numpy import genfromtxt
import matplotlib.pyplot as plt

# Import the data of a CSV file into a numpy array
cats = genfromtxt('cat.csv', delimiter=',') # 80 cats (one in a column) each by 4096 pixels
dogs = genfromtxt('dog.csv', delimiter=',') # 80 dogs (one in a column) each by 4096 pixels

# Combine two arrays (cats and dogs) into one (pets)
pets = np.empty([160, 4096]) # empty array of size 160 x 4096
for m in range(80):
    pets[m,:] = np.reshape(np.transpose(np.reshape(cats[:,m], (64,64))), (1,4096)) # cats in the first 80 rows (one in a row)
    pets[m+80,:] = np.reshape(np.transpose(np.reshape(dogs[:,m], (64,64))), (1,4096)) # dogs in the other 80 rows (one in a row)

In [7]:
def ShowBarPlot(init_data, labels):
    
    import numpy as np
    import matplotlib.pyplot as plt
    
    # Converting data using edge detection
    from skimage.filters import sobel
    edge_data = np.empty([160, 4096])
    for m in range(160):
        edge_data[m,:] = np.reshape(sobel(np.reshape(init_data[m,:], (64,64))), (1,4096))
        
    # Converting data using thresholding
    import cv2
    thresh_data = np.empty([160, 4096])
    for m in range(160):
        retval,img = cv2.threshold(np.reshape(init_data[m,:], (64,64)), 135, 255, cv2.THRESH_BINARY)
        thresh_data[m,:] = np.reshape(img, (1,4096))
        
    def ClassifyData(data):
    
        # data and labels for training
        train_data = np.concatenate((data[:50, :], data[80:130,:]), axis=0)
        train_labels = np.concatenate((labels[:50], labels[80:130]), axis=0)
        # data and labels for testing
        test_data = np.concatenate((data[50:80, :], data[130:,:]), axis=0)
        test_labels = np.concatenate((labels[50:80], labels[130:]), axis=0)

        # Apply ML algorithms

        scores = []
        
        # Linear Discriminant Analysis (LDA)
        from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
        lda = LDA()
        clf = lda.fit(train_data, train_labels)
        scores.append(clf.score(test_data, test_labels))

        # Support Vector Machine (SVM)
        from sklearn import svm
        clf = svm.SVC(kernel='poly')
        clf.fit(train_data, train_labels)
        scores.append(clf.score(test_data, test_labels))
        
        # Gaussian Naive Bayes (GNB)
        from sklearn.naive_bayes import GaussianNB
        clf = GaussianNB()
        clf.fit(train_data, train_labels)
        scores.append(clf.score(test_data, test_labels))
        
        return [i * 100 for i in scores]
    
    scores_init = ClassifyData(init_data)
    scores_edge = ClassifyData(edge_data)
    scores_thresh = ClassifyData(thresh_data)
    
    algorithms = ('LDA', 'SVM', 'GNB')
    n_algs = len(algorithms)
    
    # create a plot
    fig, ax = plt.subplots()
    index = np.arange(n_algs)
    bar_width = 0.25
    opacity = 0.8

    rects1 = plt.bar(index, scores_init, bar_width, alpha=opacity, color='b', label='Initial dat')
    rects2 = plt.bar(index + bar_width, scores_edge, bar_width, alpha=opacity, color='r', label='Edge detection')
    rects3 = plt.bar(index + 2*bar_width, scores_thresh, bar_width, alpha=opacity, color='g', label='Thresholding')
    
    plt.ylabel('Accuracy (%)')
    plt.title('Data classification')
    plt.xticks(index + bar_width, algorithms)
    plt.legend()

    plt.tight_layout()
    plt.show()

In [8]:
%matplotlib
plt.close('all')

labels = np.concatenate((np.ones(80), [2] * 80), axis=0)
ShowBarPlot(pets, labels)

Using matplotlib backend: Qt5Agg
